In [79]:
from grafos import Accion, Estado, Problema, Nodo

In [80]:
def crea_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    raiz.costo = 0
    raiz.heuristicas = problema.heuristicas[estado_raiz.nombre]
    raiz.valores = dict(raiz.heuristicas.items())
    return raiz

def crea_nodo_hijo(problema, padre, accion, agregar = True):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    hijo.heuristicas = problema.heuristicas[hijo.estado.nombre]
    hijo.valores = {estado: hijo.costo - heuristica for estado, heuristica in hijo.heuristicas.items()}
    if agregar:
        padre.hijos.append(hijo)
    return hijo

def sacar_siguiente(frontera, metrica = 'valor', criterio = 'menor', objetivos = None):
    mejor = frontera[0]
    for nodo in frontera[1:]:
        for objetivo in objetivos:
            if metrica == 'valor':
                valor_nodo = nodo.valores[objetivo.nombre]
                valor_mejor = mejor.valores[objetivo.nombre]
                if(criterio == 'menor' and
                   valor_nodo < valor_mejor):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     valor_nodo > valor_mejor):
                    mejor = nodo
            elif metrica == 'heuristica':
                heuristica_nodo = nodo.heuristicas[objetivo.nombre]
                heuristica_mejor = mejor.heuristicas[objetivo.nombre]
                if(criterio == 'menor' and
                   heuristica_nodo < heuristica_mejor):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     heuristica_nodo > heuristica_mejor):
                    mejor = nodo
            elif metrica == 'costo':
                if(criterio == 'menor' and
                   nodo.costo_camino < mejor.costo_camino):
                    mejor = nodo
                elif(criterio == 'mayor' and
                     nodo.costo_camino > mejor.costo_camino):
                    mejor = nodo
    frontera.remove(mejor)
    return mejor

def muestra_solucion(objetivo = None, problema_resolver = None):
    if not objetivo:
        print("No hay solución")
        return
    nodo = objetivo
    while nodo:
        msg = "Estado {0}, Valor {1}"
        estado = nodo.estado.nombre
        valores = [nodo.valores[objetivo.nombre]
                   for objetivo
                   in problema_resolver.estado_objetivo]
        valor = min(valores)
        print(msg.format(estado, valor))
        msg = "  Costo: {0}"
        costo_total = nodo.costo
        print(msg.format(costo_total))
        msg = "  Heurística: {0}"
        heuristicas_objetivos = [nodo.heuristicas[objetivo.nombre]
                                 for objetivo
                                 in problema_resolver.estado_objetivo]
        heuristica = min(heuristicas_objetivos)
        print(msg.format(heuristica))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            if accion:
                msg = "<--- {0} [{1}] ---"
                print(msg.format(accion, costo))
        nodo = nodo.padre

In [81]:
def a_estrella(problema):
    raiz = crea_nodo_raiz(problema)
    frontera = [raiz, ]
    explorados = set()
    while True:
        if not frontera:
            return None
        nodo = sacar_siguiente(frontera, 'valor',
                               objetivos=problema.estado_objetivo)
        if problema.es_objetivo(nodo.estado):
            return nodo
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crea_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if hijo.estado in explorados or hijo.estado in estados_frontera:
                buscar = [nodo for nodo in frontera
                          if nodo.estado == hijo.estado]
                if buscar:
                    valores_hijo = [hijo.valores[objetivo.nombre]
                                    for objetivo
                                    in problema.estado_objetivo]
                    valores_buscar = [buscar[0].valores[objetivo.nombre]
                                      for objetivo
                                      in problema.estado_objetivo]
                    minimo_hijo = min(valores_hijo)
                    minimo_buscar = min(valores_buscar)
                    if minimo_hijo < minimo_buscar:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            else:
                frontera.append(hijo)

In [82]:
if __name__ == '__main__':
    accN = Accion("N")
    accS = Accion("S")
    accE = Accion("E")
    accO = Accion("O")
    accNE = Accion("NE")
    accNO = Accion("NO")
    accSE = Accion("SE")
    accSO = Accion("SO")

    manizales = Estado("Manizales", [accN])
    medellin = Estado("Medellín", [accN])
    santa_fe = Estado("Santa Fé de Antioquia", [accN])
    dabeiba = Estado("Dabeiba", [accN])
    chigorodo = Estado("Chigorodó", [accN])
    arboletes = Estado("Arboletes", [accN])
    barbosa = Estado("Barbosa", [accN])
    yarumal = Estado("Yarumal", [accN])
    cisneros = Estado("Cisneros", [accN])
    amalfi = Estado("Amalfi", [accN])
    maceo = Estado("Maceo", [accN])
    remedios = Estado("Remedios", [accN])
    caucasia = Estado("Caucasia", [accN])
    monteria = Estado("Montería", [accN])
    planeta_rica = Estado("Planeta Rica", [accN])
    shagun = Estado("Shagún", [accN])
    cienaga_de_oro = Estado("Ciénaga de Oro", [accN])
    cerete = Estado("Cereté", [accN])
    covenas = Estado("Coveñas", [accN])
    sincelejo = Estado("Sincelejo", [accN])
    tolu = Estado("Tolú", [accN])

    acciones = {"Manizales": {"N": medellin},
                "Medellín": {"NO": santa_fe, "NE": barbosa},
                "Santa Fé de Antioquia": {"NO": dabeiba, "SE": medellin},
                "Dabeiba": {"NO": chigorodo, "SE": santa_fe},
                "Chigorodó": {"N": arboletes, "SE": dabeiba},
                "Arboletes": {"E": monteria, "S": chigorodo},
                "Barbosa": {"N": yarumal, "E": cisneros, "SO": medellin},
                "Yarumal": {"NE": caucasia, "E": amalfi, "S": barbosa},
                "Cisneros": {"E": maceo, "O": barbosa},
                "Amalfi": {"E": remedios, "O": yarumal},
                "Maceo": {"N": remedios, "O": cisneros},
                "Remedios": {"NO": caucasia, "O": amalfi, "S": maceo},
                "Caucasia": {"NO": planeta_rica, "SO": yarumal, "SE": remedios},
                "Montería": {"NE": cerete, "O": arboletes, "SE": planeta_rica},
                "Planeta Rica": {"NO": monteria, "N": shagun, "SE": caucasia},
                "Shagún": {"NE": sincelejo, "SO": cienaga_de_oro, "S": planeta_rica},
                "Ciénaga de Oro": {"O": monteria, "NE": shagun},
                "Cereté": {"E": cienaga_de_oro, "NE": covenas, "SO": monteria},
                "Coveñas": {"SO": cerete, "NE": tolu},
                "Sincelejo": {"NO": tolu, "SO": shagun},
                "Tolú": {"SO": covenas, "SE": sincelejo}
                }
    
    costos = {"Manizales": {"N": 228},
                "Medellín": {"NO": 56, "NE": 42},
                "Santa Fé de Antioquia": {"NO": 103, "SE": 56},
                "Dabeiba": {"NO": 109, "SE": 103},
                "Chigorodó": {"N": 177, "SE": 109},
                "Arboletes": {"E": 120, "S": 177},
                "Barbosa": {"N": 99, "E": 38, "SO": 42},
                "Yarumal": {"NE": 161, "E": 157, "S": 99},
                "Cisneros": {"E": 48, "O": 38},
                "Amalfi": {"E": 77, "O": 157},
                "Maceo": {"N": 83, "O": 48},
                "Remedios": {"NO": 148,"O": 77, "S": 83},
                "Caucasia": {"NO": 70, "SO": 161, "SE": 148},
                "Montería": {"NE": 23, "O": 120, "SE": 54},
                "Planeta Rica": {"NO": 54, "N": 73, "SE": 70},
                "Shagún": {"NE": 48, "SO": 34, "S": 73},
                "Ciénaga de Oro": {"O": 19, "NE": 34},
                "Cereté": {"E": 19, "NE": 73, "SO": 23},
                "Coveñas": {"SO": 73, "NE": 22},
                "Sincelejo": {"NO": 40, "SO": 48},
                "Tolú": {"SO": 22, "SE": 40}
                }
    
#     heuristicas = {
#         'Manizales': {'Manizales': 0, 'Medellín': 90, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Medellín': {'Manizales': 90, 'Medellín': 0, 'Santa Fé de Antioquia': 40, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 40, 'Yarumal': 40, 'Cisneros': 70, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Santa Fé de Antioquia': {'Manizales': 0, 'Medellín': 40, 'Santa Fé de Antioquia': 0, 'Dabeiba': 70, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Dabeiba': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 70, 'Dabeiba': 0, 'Chigorodó': 90, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Chigorodó': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 90, 'Chigorodó': 0, 'Arboletes': 70, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Arboletes': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 70, 'Arboletes': 0, 'Montería': 40, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Barbosa': {'Manizales': 0, 'Medellín': 40, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Yarumal': {'Manizales': 0, 'Medellín': 40, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 40, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 40, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Cisneros': {'Manizales': 0, 'Medellín': 70, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 70, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Maceo': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 70, 'Maceo': 0, 'Remedios': 40, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Remedios': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 40, 'Remedios': 0, 'Amalfi': 40, 'Caucasia': 90, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Amalfi': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 40, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 40, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Caucasia': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 70, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 90, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 90, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Montería': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 40, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 70, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 40, 'Covenas': 0, 'Tolú': 0},
#         'Planeta Rica': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 70, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 90, 'Planeta Rica': 0, 'Shagún': 40, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Shagún': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 40, 'Shagún': 0, 'Sincelejo': 90, 'Ciénaga de Oro': 70, 'Cereté': 0, 'Covenas': 0, 'Tolú': 0},
#         'Sincelejo': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 90, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 0, 'Tolú': 70},
#         'Ciénaga de Oro': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 70, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 40, 'Covenas': 0, 'Tolú': 0},
#         'Cereté': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 40, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 40, 'Cereté': 0, 'Covenas': 90, 'Tolú': 0},
#         'Covenas': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 0, 'Ciénaga de Oro': 0, 'Cereté': 90, 'Covenas': 0, 'Tolú': 70},
#         'Tolú': {'Manizales': 0, 'Medellín': 0, 'Santa Fé de Antioquia': 0, 'Dabeiba': 0, 'Chigorodó': 0, 'Arboletes': 0, 'Montería': 0, 'Barbosa': 0, 'Yarumal': 0, 'Cisneros': 0, 'Maceo': 0, 'Remedios': 0, 'Amalfi': 0, 'Caucasia': 0, 'Planeta Rica': 0, 'Shagún': 0, 'Sincelejo': 70, 'Ciénaga de Oro': 0, 'Cereté': 0, 'Covenas': 70, 'Tolú': 0}
# }
    

    heuristicas = {
                "Manizales": {"Tolú": 90},
                "Medellín": {"Tolú": 90},
                "Santa Fé de Antioquia": {"Tolú": 40},
                "Dabeiba": {"Tolú": 70},
                "Chigorodó": {"Tolú": 90},
                "Arboletes": {"Tolú": 70},
                "Barbosa": {"Tolú": 40},
                "Yarumal": {"Tolú": 40},
                "Cisneros": {"Tolú": 70},
                "Amalfi": {"Tolú": 40},
                "Maceo": {"Tolú": 70},
                "Remedios": {"Tolú": 90},
                "Caucasia": {"Tolú": 70},
                "Montería": {"Tolú": 40},
                "Planeta Rica": {"Tolú": 90},
                "Shagún": {"Tolú": 40},
                "Ciénaga de Oro": {"Tolú": 70},
                "Cereté": {"Tolú": 40},
                "Coveñas": {"Tolú": 70},
                "Sincelejo": {"Tolú": 70},
                "Tolú": {"Tolú": 0}
                }
    

    objetivo_1 = [tolu]
    problema_1 = Problema(manizales, objetivo_1, acciones, costos, heuristicas)

    # objetivo_2 = [goorum]
    # problema_2 = Problema(lanoi, objetivo_2, acciones, costos)

    # objetivo_3 = [boomon, goorum]
    # problema_3 = Problema(lanoi, objetivo_3, acciones, costos)

In [83]:
problema_resolver = problema_1
solucion = a_estrella(problema_resolver)
muestra_solucion(solucion, problema_resolver)

Estado Tolú, Valor 761
  Costo: 761
  Heurística: 0
<--- NO [40] ---
Estado Sincelejo, Valor 651
  Costo: 721
  Heurística: 70
<--- NE [48] ---
Estado Shagún, Valor 633
  Costo: 673
  Heurística: 40
<--- N [73] ---
Estado Planeta Rica, Valor 510
  Costo: 600
  Heurística: 90
<--- NO [70] ---
Estado Caucasia, Valor 460
  Costo: 530
  Heurística: 70
<--- NE [161] ---
Estado Yarumal, Valor 329
  Costo: 369
  Heurística: 40
<--- N [99] ---
Estado Barbosa, Valor 230
  Costo: 270
  Heurística: 40
<--- NE [42] ---
Estado Medellín, Valor 138
  Costo: 228
  Heurística: 90
<--- N [228] ---
Estado Manizales, Valor 90
  Costo: 0
  Heurística: 90
